<a href="https://colab.research.google.com/github/PavelKrinitsin/Homework2/blob/main/%D0%94%D0%97_%E2%84%962.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задача бинарной классификации. Предсказание наиболее популярной категории качества

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [3]:
data = pd.read_csv("winequalityN.csv", index_col=0)
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
type,,,,,,,,,,,,
white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
data["target"] = (data["quality"] == 5).astype(int).replace({0: -1})

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6497 entries, white to red
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6487 non-null   float64
 1   volatile acidity      6489 non-null   float64
 2   citric acid           6494 non-null   float64
 3   residual sugar        6495 non-null   float64
 4   chlorides             6495 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6488 non-null   float64
 9   sulphates             6493 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  target                6497 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 968.6+ KB


In [13]:
df_model = data.dropna(axis=0)

In [14]:
df_model

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,target
type,,,,,,,,,,,,,
white,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6,-1
white,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6,-1
white,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6,-1
white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,-1
white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
red,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6,-1
red,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,1
red,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,-1


Перейдём к задаче бинарной классификации и будем предсказывать только наиболее популярную категорию качества. Закодируем столбец quality так, чтобы наиболее частая категория (это категория 5) получила метку 1, а все прочие категории – метку -1.

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    df_model.drop(columns=["target", "quality"]), df_model["target"], test_size=0.3
)

# Отмасштабируйте признаки
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

Разделим признаки и целевую переменную.

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

При помощи кросс-валидации подберем оптимальные значения коэффициентов регуляризации для логистической регрессии и SVM с линейным ядром.

In [20]:
scores_lr = []
scores_svm = []

for c in np.arange(0.1, 10, 1):
    lr = LogisticRegression(C=c)
    svm = SVC(C=c)

    lr.fit(X_train, y_train)
    svm.fit(X_train, y_train)

    pred_lr = lr.predict(X_test)
    pred_svm = svm.predict(X_test)

    scores_lr.append(
        {"acc": accuracy_score(y_test, pred_lr), "f1": f1_score(y_test, pred_lr)}
    )
    scores_svm.append(
        {"acc": accuracy_score(y_test, pred_svm), "f1": f1_score(y_test, pred_svm)}
    )

In [21]:
scores_lr

[{'acc': 0.7426508509541001, 'f1': 0.5564444444444444},
 {'acc': 0.7436823104693141, 'f1': 0.5613415710503089},
 {'acc': 0.7436823104693141, 'f1': 0.5613415710503089},
 {'acc': 0.7436823104693141, 'f1': 0.5613415710503089},
 {'acc': 0.7436823104693141, 'f1': 0.5613415710503089},
 {'acc': 0.7436823104693141, 'f1': 0.5613415710503089},
 {'acc': 0.7436823104693141, 'f1': 0.5613415710503089},
 {'acc': 0.7436823104693141, 'f1': 0.5613415710503089},
 {'acc': 0.7436823104693141, 'f1': 0.5613415710503089},
 {'acc': 0.7436823104693141, 'f1': 0.5613415710503089}]

In [22]:
scores_svm

[{'acc': 0.7333677153171738, 'f1': 0.47512690355329945},
 {'acc': 0.7493553378029912, 'f1': 0.5706713780918727},
 {'acc': 0.7467766890149562, 'f1': 0.5785407725321889},
 {'acc': 0.7498710675605983, 'f1': 0.5829750644883921},
 {'acc': 0.7514182568334193, 'f1': 0.5859106529209622},
 {'acc': 0.7524497163486333, 'f1': 0.5897435897435896},
 {'acc': 0.7560598246518824, 'f1': 0.5960717335610589},
 {'acc': 0.7565755544094894, 'f1': 0.6006768189509306},
 {'acc': 0.7565755544094894, 'f1': 0.6046901172529313},
 {'acc': 0.7576070139247034, 'f1': 0.6070234113712374}]